In [2]:
import tensorflow as tf
import pandas as pd
from IPython.display import clear_output

In [3]:
CSV_COLUMN_NAMES = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth', 'Species']
SPECIES = ['Setosa', 'Versicolor', 'Virginica']
train_path = tf.keras.utils.get_file(
    "iris_training.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv")
test_path = tf.keras.utils.get_file(
    "iris_test.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_test.csv")
train = pd.read_csv(train_path,names=CSV_COLUMN_NAMES,header=0)
test = pd.read_csv(test_path,names=CSV_COLUMN_NAMES,header=0)
y_train = train.pop('Species')
y_test = test.pop('Species')
# train.head()


In [4]:
Feature_Columns = []
for i in train.keys():
    Feature_Columns.append(tf.feature_column.numeric_column(i))
# print(Feature_Columns)

In [5]:
def make_input(data,labels,shuffle=True,batch_size=256):
    ds = tf.data.Dataset.from_tensor_slices((dict(data),labels))
    if shuffle:
        ds = ds.shuffle(1000).repeat()
    ds = ds.batch(batch_size)
    return ds


In [6]:
classifier = tf.estimator.DNNClassifier(feature_columns=Feature_Columns,n_classes=3,hidden_units=[30,10])
classifier.train(input_fn=lambda: make_input(train,y_train),steps=9000)
clear_output()

In [7]:
eval = classifier.evaluate(input_fn=lambda : make_input(test,y_test,shuffle=False))
clear_output()
print(eval['accuracy'])

0.9


In [ ]:
def make_input_predict(data,batch_size=256):
    ds = tf.data.Dataset.from_tensor_slices((dict(data))).batch(batch_size=batch_size)
    return ds


features = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth']
predict = {}
for i in features:
    isValid = True
    while isValid:
        val = input(i)
        if val.isdigit():isValid=False
    predict[i] = [float(val)]

predict_x = {
    'SepalLength': [5.1, 5.9, 6.9],
    'SepalWidth': [3.3, 3.0, 3.1],
    'PetalLength': [1.7, 4.2, 5.4],
    'PetalWidth': [0.5, 1.5, 2.1],
}


predictions = classifier.predict(input_fn=lambda: make_input_predict(predict))
clear_output()

for i in predictions:
    class_id = i['class_ids'][0]
    probability = i['probabilities'][class_id]
    print(f"{features[class_id]} => {probability*100}")
